In [0]:
import torch
try:
    import torchbearer
except:
    !pip install torchbearer
    import torchbearer

from torchbearer import Trial
from torchbearer import callbacks
from torch import nn
import torch.nn.functional as F
from torchbearer.callbacks.torch_scheduler import LambdaLR
from torchbearer.callbacks import Callback
import numpy as np
import h5py
from torch.utils.data import Dataset, DataLoader
import os
from time import time
import sys
import glob
import h5py
import json

In [0]:
def knn(x, k):
    inner = -2*torch.matmul(x.transpose(2, 1), x)
    xx = torch.sum(x**2, dim=1, keepdim=True)
    pairwise_distance = -xx - inner - xx.transpose(2, 1)
 
    idx = pairwise_distance.topk(k=k, dim=-1)[1]   # (batch_size, num_points, k)
    return idx


def get_graph_feature(x, k=20, idx=None, dim6=True):
    batch_size = x.size(0)
    num_points = x.size(2)
    x = x.view(batch_size, -1, num_points)
    if idx is None:
        if dim6 == False:
            idx = knn(x, k=k)   # (batch_size, num_points, k)
        else:
            idx = knn(x[:, :3], k=k)
    device = torch.device('cuda')

    idx_base = torch.arange(0, batch_size, device=device).view(-1, 1, 1)*num_points

    idx = idx + idx_base

    idx = idx.view(-1)
 
    _, num_dims, _ = x.size()

    x = x.transpose(2, 1).contiguous()   # (batch_size, num_points, num_dims)  -> (batch_size*num_points, num_dims) #   batch_size * num_points * k + range(0, batch_size*num_points)
    feature = x.view(batch_size*num_points, -1)[idx, :]
    feature = feature.view(batch_size, num_points, k, num_dims) 
    x = x.view(batch_size, num_points, 1, num_dims).repeat(1, 1, k, 1)
    
    feature = torch.cat((feature-x, x), dim=3).permute(0, 3, 1, 2)
  
    return feature      # (batch_size, 2*num_dims, num_points, k)


In [0]:
#dgcnn for semseg

class DGCNN_semseg(nn.Module):
    def __init__(self, k=20, emb_dims=1024, dropout=0.5, seg_num_all=27):
        super(DGCNN_semseg, self).__init__()
        self.k = k
        
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(64)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(64)
        self.bn5 = nn.BatchNorm2d(64)
        self.bn6 = nn.BatchNorm1d(emb_dims)
        self.bn7 = nn.BatchNorm1d(512)
        self.bn8 = nn.BatchNorm1d(256)

        self.conv1 = nn.Sequential(nn.Conv2d(12, 64, kernel_size=1, bias=False),
                                   self.bn1,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1, bias=False),
                                   self.bn2,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv3 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn3,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv4 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=1, bias=False),
                                   self.bn4,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv5 = nn.Sequential(nn.Conv2d(64*2, 64, kernel_size=1, bias=False),
                                   self.bn5,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv6 = nn.Sequential(nn.Conv1d(192, emb_dims, kernel_size=1, bias=False),
                                   self.bn6,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv7 = nn.Sequential(nn.Conv1d(1216, 512, kernel_size=1, bias=False),
                                   self.bn7,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.conv8 = nn.Sequential(nn.Conv1d(512, 256, kernel_size=1, bias=False),
                                   self.bn8,
                                   nn.LeakyReLU(negative_slope=0.2))
        self.dp1 = nn.Dropout(p=dropout)
        self.conv9 = nn.Conv1d(256, seg_num_all, kernel_size=1, bias=False)
        

    def forward(self, x):
        batch_size = x.size(0)
        num_points = x.size(2)

        x = get_graph_feature(x, k=self.k)   # (batch_size, 9, num_points) -> (batch_size, 9*2, num_points, k)
        x = self.conv1(x)                       # (batch_size, 9*2, num_points, k) -> (batch_size, 64, num_points, k)
        x = self.conv2(x)                       # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points, k)
        x1 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x1, k=self.k)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv3(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x = self.conv4(x)                       # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points, k)
        x2 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = get_graph_feature(x2, k=self.k)     # (batch_size, 64, num_points) -> (batch_size, 64*2, num_points, k)
        x = self.conv5(x)                       # (batch_size, 64*2, num_points, k) -> (batch_size, 64, num_points, k)
        x3 = x.max(dim=-1, keepdim=False)[0]    # (batch_size, 64, num_points, k) -> (batch_size, 64, num_points)

        x = torch.cat((x1, x2, x3), dim=1)      # (batch_size, 64*3, num_points)

        x = self.conv6(x)                       # (batch_size, 64*3, num_points) -> (batch_size, emb_dims, num_points)
        x = x.max(dim=-1, keepdim=True)[0]      # (batch_size, emb_dims, num_points) -> (batch_size, emb_dims, 1)

        x = x.repeat(1, 1, num_points)          # (batch_size, 1024, num_points)
        x = torch.cat((x, x1, x2, x3), dim=1)   # (batch_size, 1024+64*3, num_points)

        x = self.conv7(x)                       # (batch_size, 1024+64*3, num_points) -> (batch_size, 512, num_points)
        x = self.conv8(x)                       # (batch_size, 512, num_points) -> (batch_size, 256, num_points)
        x = self.dp1(x)
        x = self.conv9(x)                       # (batch_size, 256, num_points) -> (batch_size, seg_num_all, num_points)
        
        return x.permute(0,2,1)

In [0]:
def loss_function(pred, gold, smoothing=True):
    ''' Calculate cross entropy loss, apply label smoothing if needed. '''
    
    pred = pred.contiguous().view(-1, pred.shape[-1])
    gold = gold.contiguous().flatten()

    if smoothing:
        eps = 0.2
        n_class = pred.size(1)

        one_hot = torch.zeros_like(pred).scatter(1, gold.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(pred, dim=1)
        loss = -(one_hot * log_prb).sum(dim=1).mean()
    else:
        loss = F.cross_entropy(pred, gold, reduction='mean')

    return loss

In [0]:
def load_data_semseg(partition, test_area, train_area):
    DATA_DIR = '/content/drive/My Drive/deep_data/'
    if partition == 'train':
        data_dir = os.path.join(DATA_DIR, 'indoor3d_sem_seg_hdf5_data')
    else:
        data_dir = os.path.join(DATA_DIR, 'indoor3d_sem_seg_hdf5_data_test')
    with open(os.path.join(data_dir, "all_files.txt")) as f:
        all_files = [line.rstrip() for line in f]
    with open(os.path.join(data_dir, "room_filelist.txt")) as f:
        room_filelist = [line.rstrip() for line in f]     
    data_batchlist, label_batchlist = [], []
    for f in all_files:
        file = h5py.File(os.path.join(DATA_DIR, f), 'r+')
        data = file["data"][:]
        label = file["label"][:]
        data_batchlist.append(data)
        label_batchlist.append(label)
    data_batches = np.concatenate(data_batchlist, 0)
    seg_batches = np.concatenate(label_batchlist, 0)
    train_area_name = "Area_" + train_area
    test_area_name = "Area_" + test_area
    train_idxs, test_idxs = [], []
    for i, room_name in enumerate(room_filelist):
        if test_area_name in room_name:
            test_idxs.append(i)
        elif train_area_name in room_name:
            train_idxs.append(i)

    if partition == 'train':
        all_data = data_batches[train_idxs, ...]
        all_seg = seg_batches[train_idxs, ...]
    else:
        all_data = data_batches[test_idxs, ...]
        all_seg = seg_batches[test_idxs, ...]
    return all_data, all_seg

In [0]:
class S3DIS(Dataset):
    def __init__(self, num_points=4096, partition='train', test_area='5', train_area='1'):
        self.data, self.seg = load_data_semseg(partition, test_area, train_area)
        
        self.num_points = num_points
        self.partition = partition

    def __getitem__(self, item):
        pointcloud = self.data[item][:self.num_points]
        seg = self.seg[item][:self.num_points]
        if self.partition == 'train':
            indices = list(range(pointcloud.shape[0]))
            np.random.shuffle(indices)
            pointcloud = pointcloud[indices]
            seg = seg[indices]
        seg = torch.LongTensor(seg)
        pc = torch.as_tensor(pointcloud[...,:6])
        pc = pc.permute(1,0)
        return pc, seg

    def __len__(self):
        return self.data.shape[0]

In [0]:
num_points = 4096
test_area = '5'
train_area = '3'

batch_size = 32
test_batch_size = 16

train_loader = DataLoader(S3DIS(partition='train', num_points=num_points, test_area=test_area, train_area=train_area), 
                        batch_size=batch_size, shuffle=True, drop_last=True)

test_loader = DataLoader(S3DIS(partition='test', num_points=num_points, test_area=test_area, train_area=train_area), 
                         batch_size=test_batch_size, shuffle=True, drop_last=False)


In [0]:
for data, labels in train_loader:
    print(data.shape, labels.shape)
    break

for data, labels in test_loader:
   print(data.shape, labels.shape)
   break

In [0]:
from torchbearer.callbacks.torch_scheduler import CosineAnnealingLR

model=DGCNN_semseg()
model.load_state_dict(torch.load('/content/drive/My Drive/deep_data/pretrained_semseg_100.weights'))
model.conv9 = nn.Conv1d(256, 13, kernel_size=1, bias=False)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epoch_num = 100


opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = CosineAnnealingLR(epoch_num, eta_min=1e-3)

trial = Trial(model, opt, loss_function, callbacks=[scheduler], metrics=['loss']).to(device)
trial.with_generators(train_loader, test_generator=test_loader)

trial.run(epochs=epoch_num)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)


torch.save(model.state_dict(),'/content/drive/My Drive/deep_data/trained_sem_seg.ckpt')

In [0]:
from torchbearer.callbacks.torch_scheduler import CosineAnnealingLR

model=DGCNN_semseg()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epoch_num = 100

opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = CosineAnnealingLR(epoch_num, eta_min=1e-3)

trial = Trial(model, opt, loss_function, callbacks=[scheduler], metrics=['loss', 'accuracy']).to(device)
trial.with_generators(train_loader, test_generator=test_loader)

trial.run(epochs=epoch_num)
results = trial.evaluate(data_key=torchbearer.TEST_DATA)
print(results)

In [0]:
from sklearn.metrics import jaccard_score

num_models = len(test_loader.dataset)
print(num_models)
num_points = 4096

y_preds = np.empty(num_models*num_points, dtype=int)
y_trues = np.empty(num_models*num_points, dtype=int)

for i,(data, labels) in enumerate(test_loader):
    batch_size = data.shape[0]
    y_pred = model(data.to('cuda'))
    y_pred = torch.flatten(y_pred.contiguous(), 0, 1)
    _, y_pred = y_pred.max(1)
    y_true = torch.flatten(labels.contiguous(), 0, 1)

    y_preds[i*32*num_points : i*32*num_points + batch_size*num_points] = y_pred.cpu().numpy()
    y_trues[i*32*num_points : i*32*num_points + batch_size*num_points] = y_true.cpu().numpy()

score = jaccard_score(y_trues, y_preds, average=None)

print(score)